In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU 
from tensorflow.python.keras.layers.convolutional import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

C:\Users\ragja\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ragja\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ragja\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ragja\Anaconda3\envs\test\lib\site-packages\tensorflow\python\frame

In [28]:
class GAN():
    def __init__(self):
        self.N = 60000
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.dropout = 0.9
        lengthscale = 1e-2
        self.reg = lengthscale**2 * (1 - self.dropout) / (2. * self.N )
        
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.generator = self.build_generator()

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        print(img.shape)

        self.discriminator.trainable = False
        validity = self.discriminator(img)
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):                
        model = Sequential()
        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D(size=(2, 2)))
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)        
        return Model(noise, img)
    
    def build_discriminator(self):
        model = Sequential()
        model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=5, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=5, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        img = Input(shape=self.img_shape)
        validity = model(img)     
        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        (X_train, _), (_, _) = cifar10.load_data()
        X_train = X_train / 127.5 - 1.
        #X_train = np.expand_dims(X_train, axis=3)
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            if epoch % sample_interval == 0:
               self.sample_images(epoch)
            
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
    
    def predict(self, no_samples):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        for k in range(no_samples):
            gen_imgs = self.generator.predict(noise)
            gen_imgs = 0.5 * gen_imgs + 0.5
            fig, axs = plt.subplots(r, c)
            cnt = 0
            for i in range(r):
                for j in range(c):
                    axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                    axs[i,j].axis('off')
                    cnt += 1
            fig.savefig("images/%d.png" % k)
            plt.close()

In [29]:
gan = GAN()
gan.train(epochs=10000, batch_size=128, sample_interval=1000)

(?, 32, 32, 3)
0 [D loss: 4.693426, acc.: 18.36%] [G loss: 0.189206]
1 [D loss: 0.582722, acc.: 67.19%] [G loss: 6.090902]
2 [D loss: 1.539060, acc.: 53.52%] [G loss: 0.453432]
3 [D loss: 0.052955, acc.: 98.05%] [G loss: 0.011505]
4 [D loss: 0.009054, acc.: 100.00%] [G loss: 0.002958]
5 [D loss: 0.004005, acc.: 100.00%] [G loss: 0.001075]
6 [D loss: 0.003608, acc.: 100.00%] [G loss: 0.000693]
7 [D loss: 0.002458, acc.: 100.00%] [G loss: 0.000768]
8 [D loss: 0.003044, acc.: 100.00%] [G loss: 0.000440]
9 [D loss: 0.002769, acc.: 100.00%] [G loss: 0.000646]
10 [D loss: 0.003787, acc.: 100.00%] [G loss: 0.000753]
11 [D loss: 0.006409, acc.: 100.00%] [G loss: 0.000962]
12 [D loss: 0.016344, acc.: 100.00%] [G loss: 0.000741]
13 [D loss: 0.012398, acc.: 99.61%] [G loss: 0.001048]
14 [D loss: 0.005617, acc.: 100.00%] [G loss: 0.002432]
15 [D loss: 0.014099, acc.: 100.00%] [G loss: 0.005917]
16 [D loss: 0.022860, acc.: 99.61%] [G loss: 0.012688]
17 [D loss: 0.009863, acc.: 100.00%] [G loss: 0.0

21 [D loss: 5.117842, acc.: 30.86%] [G loss: 0.450117]
22 [D loss: 1.036488, acc.: 67.19%] [G loss: 0.000659]
23 [D loss: 0.178080, acc.: 94.53%] [G loss: 0.000009]
24 [D loss: 0.073627, acc.: 97.27%] [G loss: 0.000000]
25 [D loss: 0.073299, acc.: 98.44%] [G loss: 0.000417]
26 [D loss: 0.007110, acc.: 100.00%] [G loss: 0.045351]
27 [D loss: 0.040392, acc.: 98.05%] [G loss: 0.001550]
28 [D loss: 0.009878, acc.: 99.61%] [G loss: 0.001268]
29 [D loss: 0.116947, acc.: 95.70%] [G loss: 9.390232]
30 [D loss: 0.075783, acc.: 97.66%] [G loss: 7.741568]
31 [D loss: 6.577467, acc.: 46.09%] [G loss: 16.118095]
32 [D loss: 3.495406, acc.: 53.12%] [G loss: 13.202225]
33 [D loss: 0.141465, acc.: 96.09%] [G loss: 1.585538]
34 [D loss: 4.105141, acc.: 50.00%] [G loss: 9.940357]
35 [D loss: 1.070371, acc.: 80.86%] [G loss: 8.707846]
36 [D loss: 0.893895, acc.: 74.61%] [G loss: 2.380559]
37 [D loss: 1.017679, acc.: 59.38%] [G loss: 6.109340]
38 [D loss: 0.720901, acc.: 74.22%] [G loss: 6.235332]
39 [D l

43 [D loss: 0.816734, acc.: 64.45%] [G loss: 1.375471]
44 [D loss: 0.569263, acc.: 75.00%] [G loss: 2.502732]
45 [D loss: 0.455137, acc.: 82.81%] [G loss: 1.963102]
46 [D loss: 0.516819, acc.: 77.34%] [G loss: 1.434399]
47 [D loss: 0.700386, acc.: 70.70%] [G loss: 2.165159]
48 [D loss: 0.918656, acc.: 61.72%] [G loss: 3.237634]
49 [D loss: 0.899705, acc.: 64.84%] [G loss: 2.012882]
50 [D loss: 0.958119, acc.: 55.86%] [G loss: 1.923897]
51 [D loss: 1.118532, acc.: 45.31%] [G loss: 2.120919]
52 [D loss: 0.632823, acc.: 73.05%] [G loss: 1.798376]
53 [D loss: 0.519468, acc.: 75.00%] [G loss: 1.315235]
54 [D loss: 0.549780, acc.: 77.73%] [G loss: 0.910254]
55 [D loss: 0.227437, acc.: 92.19%] [G loss: 0.899843]
56 [D loss: 0.222244, acc.: 91.80%] [G loss: 0.574461]
57 [D loss: 0.196074, acc.: 96.09%] [G loss: 0.420782]
58 [D loss: 0.202890, acc.: 94.53%] [G loss: 0.854596]
59 [D loss: 0.366506, acc.: 86.72%] [G loss: 1.354200]
60 [D loss: 0.732708, acc.: 63.67%] [G loss: 2.043644]
61 [D loss

65 [D loss: 0.427876, acc.: 77.34%] [G loss: 1.623417]
66 [D loss: 0.215912, acc.: 92.58%] [G loss: 0.418956]
67 [D loss: 0.124532, acc.: 96.48%] [G loss: 0.219755]
68 [D loss: 0.183253, acc.: 95.31%] [G loss: 0.285177]
69 [D loss: 0.080562, acc.: 96.88%] [G loss: 0.157818]
70 [D loss: 0.131000, acc.: 97.27%] [G loss: 0.198448]
71 [D loss: 0.119450, acc.: 96.09%] [G loss: 0.452519]
72 [D loss: 0.088535, acc.: 98.05%] [G loss: 0.127480]
73 [D loss: 0.092693, acc.: 98.05%] [G loss: 0.267884]
74 [D loss: 0.169747, acc.: 94.92%] [G loss: 0.288280]
75 [D loss: 0.075816, acc.: 97.66%] [G loss: 0.249255]
76 [D loss: 0.027747, acc.: 100.00%] [G loss: 0.233752]
77 [D loss: 0.262441, acc.: 91.41%] [G loss: 1.680817]
78 [D loss: 0.149401, acc.: 95.31%] [G loss: 0.099455]
79 [D loss: 0.188058, acc.: 96.09%] [G loss: 0.085256]
80 [D loss: 0.055549, acc.: 99.61%] [G loss: 0.116062]
81 [D loss: 0.135673, acc.: 97.27%] [G loss: 0.381534]
82 [D loss: 0.537673, acc.: 73.44%] [G loss: 8.174828]
83 [D los

87 [D loss: 1.323829, acc.: 62.11%] [G loss: 2.624774]
88 [D loss: 0.085479, acc.: 98.05%] [G loss: 0.677190]
89 [D loss: 0.078167, acc.: 97.66%] [G loss: 0.845852]
90 [D loss: 0.039790, acc.: 98.83%] [G loss: 0.761670]
91 [D loss: 0.085514, acc.: 96.88%] [G loss: 0.534512]
92 [D loss: 0.022581, acc.: 99.22%] [G loss: 0.246700]
93 [D loss: 0.028971, acc.: 99.61%] [G loss: 0.207864]
94 [D loss: 0.056778, acc.: 98.83%] [G loss: 0.285825]
95 [D loss: 0.029434, acc.: 99.61%] [G loss: 0.639396]
96 [D loss: 0.035815, acc.: 99.61%] [G loss: 0.518723]
97 [D loss: 0.021716, acc.: 100.00%] [G loss: 0.367032]
98 [D loss: 0.053548, acc.: 98.83%] [G loss: 0.146593]
99 [D loss: 0.115575, acc.: 95.31%] [G loss: 2.202102]
100 [D loss: 0.075355, acc.: 97.27%] [G loss: 1.380750]
101 [D loss: 0.067359, acc.: 98.44%] [G loss: 0.016051]
102 [D loss: 0.052005, acc.: 99.22%] [G loss: 0.005618]
103 [D loss: 0.085730, acc.: 96.88%] [G loss: 0.135148]
104 [D loss: 0.211100, acc.: 91.41%] [G loss: 0.314101]
105 

...

9899 [D loss: 0.438481, acc.: 83.20%] [G loss: 2.316986]
9900 [D loss: 0.488192, acc.: 68.36%] [G loss: 1.359695]
9901 [D loss: 0.426271, acc.: 83.59%] [G loss: 0.454962]
9902 [D loss: 0.674683, acc.: 62.50%] [G loss: 1.777856]
9903 [D loss: 1.038254, acc.: 35.55%] [G loss: 1.436622]
9904 [D loss: 0.380834, acc.: 88.67%] [G loss: 1.435355]
9905 [D loss: 0.391790, acc.: 85.16%] [G loss: 0.681178]
9906 [D loss: 0.346112, acc.: 90.23%] [G loss: 0.783476]
9907 [D loss: 0.369306, acc.: 86.33%] [G loss: 0.450819]
9908 [D loss: 0.174183, acc.: 97.27%] [G loss: 0.568531]
9909 [D loss: 0.238979, acc.: 95.70%] [G loss: 0.853317]
9910 [D loss: 0.298297, acc.: 85.94%] [G loss: 0.281083]
9911 [D loss: 0.145739, acc.: 96.48%] [G loss: 0.191578]
9912 [D loss: 0.266750, acc.: 88.28%] [G loss: 0.519778]
9913 [D loss: 0.601277, acc.: 67.58%] [G loss: 0.403270]
9914 [D loss: 0.331574, acc.: 82.42%] [G loss: 0.287168]
9915 [D loss: 0.408261, acc.: 85.16%] [G loss: 0.380911]
9916 [D loss: 0.678323, acc.: 6

9921 [D loss: 0.957260, acc.: 40.23%] [G loss: 1.700650]
9922 [D loss: 1.233739, acc.: 20.31%] [G loss: 0.875396]
9923 [D loss: 0.608254, acc.: 66.02%] [G loss: 2.791871]
9924 [D loss: 0.563951, acc.: 72.66%] [G loss: 1.711883]
9925 [D loss: 0.300346, acc.: 91.41%] [G loss: 0.594635]
9926 [D loss: 0.437625, acc.: 83.59%] [G loss: 0.845028]
9927 [D loss: 0.374214, acc.: 87.11%] [G loss: 1.584983]
9928 [D loss: 0.383949, acc.: 86.72%] [G loss: 0.864622]
9929 [D loss: 0.429131, acc.: 84.38%] [G loss: 0.505310]
9930 [D loss: 0.237097, acc.: 95.31%] [G loss: 0.519588]
9931 [D loss: 0.487910, acc.: 78.52%] [G loss: 1.372683]
9932 [D loss: 0.507615, acc.: 70.70%] [G loss: 0.379159]
9933 [D loss: 0.208471, acc.: 96.09%] [G loss: 0.260464]
9934 [D loss: 0.364149, acc.: 82.42%] [G loss: 0.871523]
9935 [D loss: 0.727966, acc.: 55.86%] [G loss: 0.702590]
9936 [D loss: 2.337786, acc.: 3.52%] [G loss: 0.928335]
9937 [D loss: 0.198708, acc.: 97.27%] [G loss: 2.184796]
9938 [D loss: 0.586875, acc.: 70

9943 [D loss: 0.344452, acc.: 89.45%] [G loss: 1.301367]
9944 [D loss: 1.287473, acc.: 36.33%] [G loss: 1.524297]
9945 [D loss: 0.476832, acc.: 75.39%] [G loss: 2.296230]
9946 [D loss: 0.609449, acc.: 66.80%] [G loss: 0.764584]
9947 [D loss: 0.564825, acc.: 71.88%] [G loss: 1.051190]
9948 [D loss: 0.280761, acc.: 91.02%] [G loss: 1.319414]
9949 [D loss: 0.422033, acc.: 85.16%] [G loss: 1.035687]
9950 [D loss: 0.160729, acc.: 98.44%] [G loss: 1.054270]
9951 [D loss: 0.143356, acc.: 97.27%] [G loss: 0.225800]
9952 [D loss: 0.250981, acc.: 95.31%] [G loss: 0.184640]
9953 [D loss: 0.362228, acc.: 82.03%] [G loss: 0.520967]
9954 [D loss: 0.664858, acc.: 61.33%] [G loss: 0.667949]
9955 [D loss: 0.301118, acc.: 89.06%] [G loss: 0.298953]
9956 [D loss: 1.139547, acc.: 32.42%] [G loss: 0.424113]
9957 [D loss: 0.181462, acc.: 97.27%] [G loss: 0.427601]
9958 [D loss: 0.853505, acc.: 48.83%] [G loss: 0.976633]
9959 [D loss: 0.175780, acc.: 97.66%] [G loss: 0.920789]
9960 [D loss: 0.909119, acc.: 3

9965 [D loss: 0.837244, acc.: 50.00%] [G loss: 1.311587]
9966 [D loss: 0.832570, acc.: 53.12%] [G loss: 1.497891]
9967 [D loss: 0.857415, acc.: 44.92%] [G loss: 1.668883]
9968 [D loss: 0.413832, acc.: 80.47%] [G loss: 2.447032]
9969 [D loss: 0.814569, acc.: 46.88%] [G loss: 1.214905]
9970 [D loss: 0.272576, acc.: 94.14%] [G loss: 1.712671]
9971 [D loss: 0.340352, acc.: 89.45%] [G loss: 0.968203]
9972 [D loss: 0.659301, acc.: 58.98%] [G loss: 1.428380]
9973 [D loss: 0.501624, acc.: 73.44%] [G loss: 0.714352]
9974 [D loss: 0.457598, acc.: 82.42%] [G loss: 0.869648]
9975 [D loss: 0.733139, acc.: 51.56%] [G loss: 1.009960]
9976 [D loss: 0.154515, acc.: 98.05%] [G loss: 1.093605]
9977 [D loss: 0.398042, acc.: 78.91%] [G loss: 0.563204]
9978 [D loss: 0.336854, acc.: 88.67%] [G loss: 0.753270]
9979 [D loss: 0.544874, acc.: 74.22%] [G loss: 1.061132]
9980 [D loss: 0.500084, acc.: 78.12%] [G loss: 0.726534]
9981 [D loss: 0.669206, acc.: 61.72%] [G loss: 0.661876]
9982 [D loss: 0.251639, acc.: 9

9987 [D loss: 0.272952, acc.: 92.97%] [G loss: 0.650529]
9988 [D loss: 0.848507, acc.: 44.92%] [G loss: 0.417053]
9989 [D loss: 0.331421, acc.: 87.11%] [G loss: 0.870141]
9990 [D loss: 0.841769, acc.: 44.92%] [G loss: 0.711314]
9991 [D loss: 0.308063, acc.: 94.92%] [G loss: 0.941461]
9992 [D loss: 0.764409, acc.: 54.69%] [G loss: 0.573273]
9993 [D loss: 0.499206, acc.: 73.83%] [G loss: 0.745109]
9994 [D loss: 0.413785, acc.: 82.81%] [G loss: 0.808514]
9995 [D loss: 0.500378, acc.: 73.05%] [G loss: 1.421134]
9996 [D loss: 0.501337, acc.: 74.22%] [G loss: 1.999843]
9997 [D loss: 0.297250, acc.: 94.14%] [G loss: 1.206608]
9998 [D loss: 0.595910, acc.: 66.02%] [G loss: 0.869344]
9999 [D loss: 0.420957, acc.: 83.20%] [G loss: 1.034152]


In [30]:
gan.predict(5)